# Remove based on Color

In [3]:
import cv2
import numpy as np
import os 

def get_background_color(image_path):
    """
    Gets the background color from the top left pixel of an image.

    :param image_path: Path to the image.
    :return: HSV values of the background color.
    """
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print("Image not found or unable to read.")
        return None

    # Get the color of the top left pixel and convert to HSV
    bgr_color = image[1, 1]
    hsv_color = cv2.cvtColor(np.uint8([[bgr_color]]), cv2.COLOR_BGR2HSV)[0][0]

    return hsv_color

def remove_background(image_path, output_path):
    """
    Removes the background from an image based on the color of the top left pixel.

    :param image_path: Path to the input image.
    :param output_path: Path to save the image with the background removed.
    """
    hsv_color = get_background_color(image_path)

    if hsv_color is None:
        return

    # Define a range for the background color
    sensitivity = 15  # Adjust this value based on your requirements
    lower_color_bound = np.array([hsv_color[0] - sensitivity, hsv_color[1] - sensitivity, hsv_color[2] - sensitivity])
    upper_color_bound = np.array([hsv_color[0] + sensitivity, hsv_color[1] + sensitivity, hsv_color[2] + sensitivity])

    # Read the image
    image = cv2.imread(image_path)

    # Convert to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Create a mask for the background color
    mask = cv2.inRange(hsv, lower_color_bound, upper_color_bound)

    # Invert the mask to get the foreground
    foreground = cv2.bitwise_and(image, image, mask=~mask)

    tmp = cv2.cvtColor(foreground, cv2.COLOR_BGR2GRAY)

    _,alpha = cv2.threshold(tmp, 0,255,cv2.THRESH_BINARY)

    b,g,r = cv2.split(foreground)

    rgba = [b,g,r,alpha]

    dst = cv2.merge(rgba,4)

    # # Invert the mask to get the characters
    # mask_inv = cv2.bitwise_not(mask)

    # # Convert the original image to RGBA
    # dst = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)

    # # Apply the inverted mask to set the background as transparent
    # dst[:, :, 3] = mask_inv

    # Save the result
    cv2.imwrite(output_path, dst)

folder_path = 'SpriteSheetsToProcess'
output_path = 'ReadyToEvaluate'
for filename in os.listdir(folder_path):

    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(folder_path, filename)
        output = os.path.join(output_path, 'processed_' + filename)
        remove_background(image_path, output)


# Remove based on Contour

In [1]:
import cv2
import numpy as np
import os

def remove_background(image_path, output_path):
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print("Image not found or unable to read.")
        return

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out small contours if necessary
    contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 1000]  # adjust the threshold as needed

    # Create a mask for the contours
    mask = np.zeros_like(gray)
    for contour in contours:
        cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)

    # Create an RGBA image and set the alpha channel to the mask
    rgba = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
    rgba[..., 3] = mask

    cv2.imshow('c',mask)
    cv2.imshow('t',thresh)
    cv2.imshow('r',gray)
    cv2.waitKey(0)
    # Save the result
    cv2.imwrite(output_path, rgba)

folder_path = 'SpriteSheetsToProcess'
output_path = 'ReadyToEvaluate'
for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(folder_path, filename)
        output = os.path.join(output_path, 'processed_' + filename)
        remove_background(image_path, output)